In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm

In [11]:
data = pd.read_csv("review_full.csv")
data

,Name,Introduction,Rating,Review Num,Review
0,16th Street Baptist Church,16th Street Baptist Church is located on Sixth...,4.5,343.0,. Excellent tour! I teach this history in my c...
1,Alabama Theatre,The Alabama Theatre was established in 1927 by...,4.5,1797.0,. The show was wonerful and we really enjoyed ...
2,Ave Maria Grotto,"Located in Cullman, Alabama the Ave Maria Grot...",4.5,408.0,. Interesting place to visit. A Benedictine Br...
3,Barber Motorsports Museum & Park,The charge of admission to the facility is $15...,5.0,1279.0,. “Wow” is the word I keep thinking as I refle...
4,Bellingrath Gardens and Homes,The site of the Bellingrath Gardens and Homes ...,4.5,825.0,. Very nice place. Nothing spectacular to see....
...,...,...,...,...,...
2519,Uncle Tom's Trail,Uncle Tom's Trail in Yellowstone National Park...,4.5,556.0,. Although the trail itself to the bottom of t...
2520,Upper Geyser Basin,Upper Geyser Basin is just one of the basins v...,5.0,1343.0,. Never seen anything like it...amazing. Of Ol...
2521,West Thumb Geyser Basin,West Thumb Geyser Basin is one of the numerous...,4.5,904.0,. Even if you are feeling a bit saturated with...
2522,Wyoming State Capital,The State Capital Building of Wyoming is locat...,4.0,336.0,. We had stopped by the capitol the previous f...


In [66]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

simplified_text = []

for index, row in tqdm(data.iterrows()):
    review = data['Review'][index]
    sentences = list(filter(lambda x: len(x) > 15, sent_tokenize(review)))
    embeddings = model.encode(sentences)
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i!=j:
                sim_mat[i][j] = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1)).reshape(1)

    nx_graph = nx.from_numpy_array(sim_mat)
    try:
        scores = nx.pagerank(nx_graph, max_iter=200)
    except:
        scores = [1/(s+1) for s in range(len(sentences))]
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    out_sent = ranked_sentences[0][1]
    i = 1
    while((i < len(ranked_sentences)) and (len(tokenizer(out_sent + " " + ranked_sentences[i][1]).input_ids) < 500)):
        out_sent = out_sent + " " + ranked_sentences[i][1]
        i += 1
    
    simplified_text.append(out_sent)

data["review_simple"] = simplified_text

0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
2524it [1:05:44,  1.56s/it]


In [69]:
data.to_csv("review_simplified.csv", index = False)